In [1]:
!nvidia-smi

Mon Feb 14 17:24:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers'==4.2.2'

     |████████████████████████████████| 1.8 MB 14.5 MB/s 
     |████████████████████████████████| 2.9 MB 72.0 MB/s 
     |████████████████████████████████| 895 kB 67.2 MB/s 


In [3]:
import torch
from transformers import (
    GPT2Tokenizer, GPT2LMHeadModel, TextDataset, LineByLineTextDataset,
    DataCollatorForLanguageModeling, TrainingArguments, Trainer, get_cosine_schedule_with_warmup
)

import matplotlib.pyplot as plt
%matplotlib inline

import json
import numpy as np
import seaborn as sns
import json
import nltk


In [4]:
!wget https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json

--2022-02-14 17:24:49--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.10.86
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.10.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘personachat_self_original.json’

personachat_self_or 100%[===================>] 200.13M  34.9MB/s    in 6.4s    

2022-02-14 17:24:56 (31.4 MB/s) - ‘personachat_self_original.json’ saved [209850483/209850483]



In [18]:
bos_token = '<start>'
eos_token = '<end>'

In [5]:
with open('/content/personachat_self_original.json') as json_file:
    data = json.load(json_file)

In [19]:
chats = []
for i in range(len(data['train'])):
    personality = " ".join(data['train'][i]['personality'])
    chat = data['train'][i]['utterances'][-1]['history']
    final_text = bos_token + personality + '<start_chat>'
    for j in range(len(chat)):
        if j % 2 == 0:
            final_text += '<Human>'
        else:
            final_text += '<AI>'
        final_text += chat[j]
    chats.append(final_text + eos_token)

In [23]:
chats = " ".join(chats)

In [24]:
train_data_path = 'chats.txt'
with open(train_data_path, 'w') as t:
    t.write(chats)

# Tokenezation


In [25]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token=bos_token, eos_token=eos_token, pad_token='<|pad|>')

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [26]:
block_size = 1024

In [27]:
train_dataset = TextDataset(tokenizer=tokenizer, file_path='/content/chats.txt', block_size=block_size, overwrite_cache=True)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [28]:
len(train_dataset)

4283

# Training

In [29]:
model = GPT2LMHeadModel.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [30]:
model.cuda()
None

In [31]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [32]:
learning_rate = 2e-5
batch_size = 1
num_epochs = 4

In [33]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

seed = 567
training_args = TrainingArguments(
    output_dir='chats_output',
    overwrite_output_dir=True,
    do_train=True,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    prediction_loss_only=True,
    logging_steps=100,
    load_best_model_at_end=True,
    #save_steps=300,
    seed=seed,
    learning_rate=learning_rate
)

In [34]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

train_dataloader = trainer.get_train_dataloader()
num_train_steps = len(train_dataloader) * num_epochs
trainer.create_optimizer_and_scheduler(num_train_steps) 

trainer.lr_scheduler = get_cosine_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps=num_train_steps//10,
    num_training_steps=num_train_steps
)

Заводим!

In [35]:
trainer.train() 

Step,Training Loss
100,85.025500
200,16.056700
300,4.340300
400,3.583700
500,3.257400
600,3.096200
700,2.949600
800,2.877400
900,2.799900
1000,2.747100


TrainOutput(global_step=17132, training_loss=2.874273501014754, metrics={'train_runtime': 2159.0167, 'train_samples_per_second': 7.935, 'total_flos': 13098653262544896, 'epoch': 4.0})

# Save model

In [36]:
output_dir = '/content/chats_output'

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/jokes_output/tokenizer_config.json',
 '/content/jokes_output/special_tokens_map.json',
 '/content/jokes_output/vocab.json',
 '/content/jokes_output/merges.txt',
 '/content/jokes_output/added_tokens.json')

# Text generation


In [79]:
def generate(prompt):

    model.eval()

    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    input_ids = input_ids.to('cuda') 
    eos_id = tokenizer.encode(eos_token)[0]

    sample_outputs = model.generate(
        input_ids, 
        max_length= len(input_ids[0]) + 50, 
        min_length=len(input_ids[0]) + 30,
        do_sample=True,
        top_k=50,
        top_p=0.92, 
        temperature=0.95,
        num_beams=1,
        repition_penalty=3,
        eos_token_id=eos_id,
        num_return_sequences=1
        )
    for i, sample_output in enumerate(sample_outputs):
          sample_output = sample_output[len(input_ids[0]):]
          return tokenizer.decode(sample_output, skip_special_tokens=False)

In [ ]:
prompt = '<start>i am a musician and hope to make it big some day . i play the piano and guitar and sing . i also work as a custodian to help pay the bills . my favorite type of music to sing is folk music .<start_chat>'
while True:
    prompt += '<Human>'
    prompt += input('Input: ') + '<AI>'
    generated = generate(prompt)
    if '<Human>' in generated:
        generated = generated[: generated.find('<Human>')]
    print(f'AI: {generated}')
    prompt += generated

In [81]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
